<a href="https://colab.research.google.com/github/javierber/colab_proj_repo/blob/main/Kaggle_Fashion_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from seaborn import heatmap

from sklearn.metrics import mean_squared_error, classification_report, \
confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow.keras as keras
from keras.optimizers import adam_v2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [2]:
#  You can use this function to see how your model improves over time
def plot_history(history, metric=None):
  """plot loss and passed metric.  metric is passed as string and must match 'metric'
  argument in the compile step"""
  fig, axes = plt.subplots(2,1, figsize = (5,10))
  axes[0].plot(history.history['loss'], label = "train")
  axes[0].plot(history.history['val_loss'], label='test')
  axes[0].set_title('Loss')
  axes[0].legend()
  if metric:
    axes[1].plot(history.history[metric], label = 'train')
    axes[1].plot(history.history['val_' + metric], label = 'test')
    axes[1].set_title(metric)
    axes[1].legend()

  plt.show()

In [3]:
#Load Data
#source https://www.kaggle.com/danialk/range-of-cnns-on-fashion-mnist-dataset
batch_size = 512
train_data_df = pd.read_csv('/content/sample_data/fashion-mnist_train.csv', header=0)
#test_data = pd.read_csv('/content/sample_data/fashion-mnist_test.csv')
train_data_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,...,211,220,214,74,0,255,222,128,0,0,0,0,0,0,0,0,0,44,12,0,0,40,134,162,191,214,163,146,165,79,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,...,171,249,207,197,202,45,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,21,25,69,52,45,74,39,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,...,230,237,229,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,116,112,136,147,144,121,102,63,0,0,0,0,0,0,0,0,0,0


In [5]:
train_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 359.3 MB


In [27]:
train_data_df.duplicated().any()

True

In [4]:
X = train_data_df.drop(columns= 'label')
y = train_data_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
y_train.shape

(45000, 10)

In [8]:
#Create your model
reg_model = Sequential()
reg_model.add(Dense(10, activation = 'relu', input_dim = X_train.shape[1]))
reg_model.add(Dense(1))

In [9]:
# Compile your model
reg_model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

In [10]:
# Fit your model
history = reg_model.fit(X_train, y_train,
                        validation_data = (X_test, y_test),
                        epochs = 200)

Epoch 1/200
1407/1407 [==============================] - 4s 2ms/step - loss: 3.4904 - mae: 1.2587 - val_loss: 2.0684 - val_mae: 1.0299
Epoch 2/200
1407/1407 [==============================] - 3s 2ms/step - loss: 1.8649 - mae: 0.9585 - val_loss: 1.8183 - val_mae: 0.9275
Epoch 3/200
1407/1407 [==============================] - 3s 2ms/step - loss: 1.6940 - mae: 0.9031 - val_loss: 1.7616 - val_mae: 0.9307
Epoch 4/200
1407/1407 [==============================] - 3s 2ms/step - loss: 1.6025 - mae: 0.8737 - val_loss: 1.6915 - val_mae: 0.8753
Epoch 5/200
1407/1407 [==============================] - 3s 2ms/step - loss: 1.5390 - mae: 0.8519 - val_loss: 1.7186 - val_mae: 0.9098
Epoch 6/200
1407/1407 [==============================] - 3s 2ms/step - loss: 1.4904 - mae: 0.8392 - val_loss: 1.6997 - val_mae: 0.8841
Epoch 7/200
1407/1407 [==============================] - 3s 2ms/step - loss: 1.4477 - mae: 0.8217 - val_loss: 1.5731 - val_mae: 0.8438
Epoch 8/200
1407/1407 [==============================] 

In [39]:
import tensorflow as tf
score = reg_model(X_test, y_test)

print('Test Loss:', score[0])
print('Test ACC:', score[1])

Test Loss: tf.Tensor([7.43625], shape=(1,), dtype=float32)
Test ACC: tf.Tensor([7.2784348], shape=(1,), dtype=float32)


#CNN

In [6]:
img_rows = 28
img_cols = 28
input_shape = (img_rows, img_cols, 1)
X_train = X_train.values.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.values.reshape(X_test.shape[0], img_rows, img_cols, 1)



In [13]:
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
cnn1 = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

In [14]:
opt = keras.optimizers.Adam(learning_rate=0.01)
cnn1.compile(loss='categorical_crossentropy', optimizer=opt, mmetrics = ['mae'])

In [15]:
history = cnn1.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_test, y_test))

Epoch 1/20
88/88 [==============================] - 21s 232ms/step - loss: 30.4438 - val_loss: 0.5324
Epoch 2/20
88/88 [==============================] - 20s 229ms/step - loss: 0.4995 - val_loss: 0.4683
Epoch 3/20
88/88 [==============================] - 20s 228ms/step - loss: 0.4241 - val_loss: 0.4279
Epoch 4/20
88/88 [==============================] - 20s 227ms/step - loss: 0.3817 - val_loss: 0.3982
Epoch 5/20
88/88 [==============================] - 20s 227ms/step - loss: 0.3525 - val_loss: 0.3801
Epoch 6/20
88/88 [==============================] - 20s 227ms/step - loss: 0.3380 - val_loss: 0.3823
Epoch 7/20
88/88 [==============================] - 20s 228ms/step - loss: 0.3225 - val_loss: 0.3705
Epoch 8/20
88/88 [==============================] - 20s 227ms/step - loss: 0.3088 - val_loss: 0.3697
Epoch 9/20
88/88 [==============================] - 20s 227ms/step - loss: 0.2958 - val_loss: 0.3635
Epoch 10/20
88/88 [==============================] - 20s 228ms/step - loss: 0.2841 - val_l

In [27]:
cnn1.optimizer.lr = 0.0001

In [28]:
cnn1.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_test, y_test))

Epoch 1/10
88/88 [==============================] - 20s 228ms/step - loss: 0.1835 - val_loss: 0.3952
Epoch 2/10
88/88 [==============================] - 20s 226ms/step - loss: 0.1820 - val_loss: 0.3959
Epoch 3/10
88/88 [==============================] - 20s 228ms/step - loss: 0.1790 - val_loss: 0.3965
Epoch 4/10
88/88 [==============================] - 20s 228ms/step - loss: 0.1803 - val_loss: 0.3976
Epoch 5/10
88/88 [==============================] - 20s 227ms/step - loss: 0.1778 - val_loss: 0.3977
Epoch 6/10
88/88 [==============================] - 20s 228ms/step - loss: 0.1809 - val_loss: 0.3980
Epoch 7/10
88/88 [==============================] - 20s 228ms/step - loss: 0.1802 - val_loss: 0.3984
Epoch 8/10
88/88 [==============================] - 20s 227ms/step - loss: 0.1773 - val_loss: 0.3977
Epoch 9/10
88/88 [==============================] - 20s 228ms/step - loss: 0.1782 - val_loss: 0.3972
Epoch 10/10
88/88 [==============================] - 20s 227ms/step - loss: 0.1765 - val_lo

In [42]:
# get predictions for testing data
X_test = np.array(X_test).reshape(X_test.shape[0], 28, 28, 1)

preds = np.argmax(cnn1.predict(X_test), axis=-1)


In [65]:
score = [float(cnn1.evaluate(X_test, y_test, verbose=0))]
#acc_score = cnn1.evaluate(y_test, preds)  - getting errors for accurracy


print(f'Test loss: {score[0]*100:.2f}%')
#print('Test accuracy:', acc_score[0]*100)

Test loss: 39.80%
